<a href="https://colab.research.google.com/github/mpakinggan/ML_Projects/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.9.0-dev20220318


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-03-18 22:16:08--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.004s  

2022-03-18 22:16:08 (86.8 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2022-03-18 22:16:08--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

Resource: https://www.tensorflow.org/text/tutorials/text_classification_rnn

In [ ]:
train_dataset = pd.read_csv(
    train_file_path, 
    sep='\t', 
    header=None,
    names=['label','text'],
    usecols=['label','text'],
    dtype={'label': 'str', 'text': 'str'}
)
test_dataset = pd.read_csv(test_file_path, 
    sep='\t', 
    header=None,
    names=['label','text'],
    usecols=['label','text'],
    dtype={'label': 'str', 'text': 'str'}
)

In [ ]:
train_dataset.loc[train_dataset['label'] == 'spam', 'label'] = 1
train_dataset.loc[train_dataset['label'] == 'ham', 'label'] = 0
test_dataset.loc[test_dataset['label'] == 'spam', 'label'] = 1
test_dataset.loc[test_dataset['label'] == 'ham', 'label'] = 0
train_labels = np.array(train_dataset.pop('label').astype('int64'))
test_labels = np.array(test_dataset.pop('label').astype('int64'))

In [ ]:
# Vectorize layer for encoding
max_features = 10000
sequence_length = 250
vectorize_layer = keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)
vectorize_layer.adapt(train_dataset.values)   

In [ ]:
vocab = vectorize_layer.get_vocabulary()    # Vocabulary list

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [vocab.index(word) if word in vocab else 0 for word in tokens]
  return keras.preprocessing.sequence.pad_sequences([tokens], sequence_length)[0]

In [ ]:
train_ds = train_dataset.values.tolist()
train_ds = np.array([encode_text(sentence[0]) for sentence in train_ds], dtype='int32')

In [ ]:
vocab_size = len(vocab)
embedding_dim = 16

model = tf.keras.Sequential([
  keras.layers.Embedding(vocab_size, embedding_dim),
  tf.keras.layers.LSTM(embedding_dim),
  keras.layers.Dense(1, activation='sigmoid')]
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          130896    
                                                                 
 lstm (LSTM)                 (None, 16)                2112      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 133,025
Trainable params: 133,025
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
history = model.fit(
    train_ds,
    train_labels,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
105/105 [==============================] - 18s 174ms/step - loss: 0.0407 - acc: 0.9895 - val_loss: 0.0655 - val_acc: 0.9880
Epoch 2/10
105/105 [==============================] - 9s 89ms/step - loss: 0.0236 - acc: 0.9964 - val_loss: 0.0661 - val_acc: 0.9880
Epoch 3/10
105/105 [==============================] - 9s 90ms/step - loss: 0.0160 - acc: 0.9982 - val_loss: 0.0646 - val_acc: 0.9880
Epoch 4/10
105/105 [==============================] - 9s 89ms/step - loss: 0.0119 - acc: 0.9991 - val_loss: 0.0646 - val_acc: 0.9880
Epoch 5/10
105/105 [==============================] - 9s 90ms/step - loss: 0.0097 - acc: 0.9991 - val_loss: 0.0684 - val_acc: 0.9821
Epoch 6/10
105/105 [==============================] - 9s 89ms/step - loss: 0.0074 - acc: 0.9994 - val_loss: 0.0640 - val_acc: 0.9856
Epoch 7/10
105/105 [==============================] - 10s 91ms/step - loss: 0.0059 - acc: 0.9997 - val_loss: 0.0642 - val_acc: 0.9856
Epoch 8/10
105/105 [==============================] - 9s 90ms/step

In [ ]:
# Encode the test dataset
test_ds = test_dataset.values.tolist()
test_ds = np.array([encode_text(sentence[0]) for sentence in test_ds], dtype='int32')

model.evaluate(test_ds,test_labels)

44/44 [==============================] - 1s 17ms/step - loss: 0.0639 - acc: 0.9792


[0.0638657808303833, 0.9791666865348816]

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict(np.array([encode_text(pred_text)], dtype='int32'))[0].tolist()
  if prediction[0] > 0.5:       # spam is encoded as 1, ham is 0
    prediction.append('spam')
  else:
    prediction.append('ham')
  return prediction

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 25ms/step
[0.0004482023941818625, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 42ms/step
You passed the challenge. Great job!
